# LVM Guide Star Catalog using the new library

In [ ]:
import lvmguiding
import numpy as np
import os
import matplotlib.pyplot as plt
import time
from astropy.coordinates import SkyCoord

In [ ]:
lvmguiding.cat_full.shape

## Creating the pointing list

In [ ]:
!pwd

In [ ]:
if True: #Old way of creating the pointing list:
    pointing_list = []

    counter = 0
    ra_list =[]
    dec_list=[]
    for ra in np.arange(5,355,30):
        for dec in np.arange(-85,85,20):
            c = SkyCoord(frame='icrs',ra=ra, dec=dec,unit='deg')
            pointing_list.append(c)
            ra_list.append(ra)
            dec_list.append(dec)
    #pointing_list=pointing_list[:10]

    indices = np.array(range(len(pointing_list)))

    output = np.stack((indices,ra_list,dec_list),axis=1)

    os.system("mkdir guide_star_search_results")
    filename = "guide_star_search_results/pointing_list"

    #pointing_list = pointing_list[[1,4,34,56,70,100]]
    if True: np.savetxt(filename,output,fmt="%10.6f")

    #pointing_list = pointing_list[::50]

if False: #new way of creating the pointing list
    from astropy.io import fits
    with fits.open("lvmsurveysim_hz_1000.fits") as hdul:
        my_data = hdul[1].data

    my_data = my_data[my_data["ra"]>-500]

    ra_list = my_data["ra"]
    dec_list = my_data["dec"]

    indices = np.array(range(len(ra_list)))




    pointing_list = []

    for index in indices:
        ra = ra_list[index]
        dec = dec_list[index]

        c = SkyCoord(frame='icrs',ra=ra, dec=dec,unit='deg')
        pointing_list.append(c)

    output = np.stack((indices,ra_list,dec_list),axis=1)


    filename = "guide_star_search_results/pointing_list"

    #pointing_list = pointing_list[[1,4,34,56,70,100]]
    if False:
        np.savetxt(filename,output,fmt="%10.6f")

fig,ax = plt.subplots(figsize=(12,6))
ax.plot(ra_list,dec_list,"k.",ms=1)
ax.set_xlabel("RA")
ax.set_ylabel("DEC") 
ax.invert_xaxis()

    
pointing_touple_list = []
for index,c in enumerate(pointing_list):
    pointing_touple_list.append((index,c))

## Sequential work on the pointing list

In [ ]:
#c = SkyCoord(frame="galactic", l=280, b=0,unit='deg'
#ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,mags,culled_cat = find_guide_stars2(c,pa=0,plotflag=True,recycled_cat=culled_cat)

if False:
    for index,c in enumerate(pointing_list):
        print("Analyzing pointing {} of {} (ra: {} dec: {})".format(index+1,len(pointing_list),c.ra.deg,c.dec.deg))
        #c = SkyCoord(frame="galactic", l=280, b=0,unit='deg')
        #print(c)
        for pa in [0,60,120,180,240,300]:
            print("PA: ",pa)
            if pa==0:
                culled_cat=cat_full
            ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,mags,culled_cat = find_guide_stars2(c,pa=pa,plotflag=False,recycled_cat=culled_cat)    

            output = np.stack((ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,mags))
            filename = "guide_star_search_results/guide_stars_{:06d}_pa_{:03d}".format(index,pa)
            np.savetxt(filename,output,fmt="%10.6f")

        
#ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,mags,culled_cat = find_guide_stars2(c,pa=60,plotflag=True,recycled_cat=culled_cat)
#ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,mags, culled_cat = find_guide_stars2(c,pa=120,plotflag=True,recycled_cat=culled_cat)
#ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,culled_cat = find_guide_stars2(c,pa=180,plotflag=True,recycled_cat=culled_cat)
#ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,culled_cat = find_guide_stars2(c,pa=240,plotflag=True,recycled_cat=culled_cat)
#ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,culled_cat = find_guide_stars2(c,pa=300,plotflag=True,recycled_cat=culled_cat)

## Paralellized work on the pointing list

In [ ]:
from multiprocessing import Pool

t00 = time.time()
num_processors = 30#Create a pool of processors
p=Pool(processes = num_processors)#get them to work in parallel
output = p.map(lvmguiding.find_guide_stars_auto,[input_touple for input_touple in pointing_touple_list[:]])
p.close()
p.join()

for o in output:
    print(o)
    
t11 = time.time()

print("Total execution time: {:.1f} s".format(t11-t00))

## Making some example plots

In [ ]:
c = pointing_touple_list[50][1]

color_array=["r","g","b","r","g","b"]


fig,ax = plt.subplots(figsize=(12,12))

for index,pa in enumerate([0,60,120,180,240,300]):
        print("PA: ",pa)
        if pa==0:
            culled_cat=lvmguiding.cat_full
        ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,mags,culled_cat = lvmguiding.find_guide_stars(c,pa=pa,plotflag=False,recycled_cat=culled_cat)
        
        if pa==0:
            ax.plot(culled_cat["ra"],culled_cat["dec"],"k.",ms=1)
        ax.plot(ras,decs,".",c=color_array[index],label="PA = "+str(pa)+" deg")
        
ax.legend()

ax.set_xlabel("RA [deg]")
ax.set_ylabel("DEC [deg]")
ax.invert_xaxis()

#fig.savefig("/home/haeberle/exchange/lvm/report/position_angles.png",bbox_inches="tight",facecolor="w",edgecolor="w",dpi=200)

In [ ]:
ras,decs,dd_x_mm,dd_y_mm,chip_xxs,chip_yys,mags,culled_cat = find_guide_stars2(c,pa=0,plotflag=False,recycled_cat=culled_cat)

fig,ax = plt.subplots(figsize=(16,9))

ax.plot(ras,decs,"k.",ms=2)
ax.plot(ras[mags<15],decs[mags<15],"r.")

#ax.invert_xaxis()

fig,ax = plt.subplots(figsize=(16,9))
ax.plot(chip_xxs[mags<18],chip_yys[mags<18],"k.",ms=2)
ax.plot(chip_xxs[mags<15],chip_yys[mags<15],"r.")
